# Pistes de réflexion explorées

Dans cette première partie nous allons décrire plusieurs pistes qui ont été explorées dans le cadre de ce projet de Machine Learning. Certaines de ces pistes ont abouti à des résultats satisfaisants et ont servi à construire peu à peu les lignes de code présentes plus loin dans ce notebook. D'autres n'ont pas semblé assez intéressantes et n'ont pas été explorées plus en détails.

## Modèles spécifiques par secteurs

Après avoir observé et lancé le notebook du benchmark, nous avons assez vite pensé à créer des modèles spécifiques pour des données se ressemblant entre elles. Notre première intuition a été de créer 12 modèles pour les 12 secteurs, en imaginant que les actions d'un même secteur ont un comportement similaire.

## Modèles spécifiques par catégories, selon la quantité de données d'entraînement

## Remarques en vrac sur les datasets

- Découpage train/test : dates blabla
- Industry ou je sais plus quoi inexistante en train

## Autres classificateurs

## Clustering

# Obtention des résultats finaux

Les lignes de code suivantes permettent de recréer notre soumission à 51,82%. C'est la solution que nous avons retenue et qui a fourni notre 3e score le plus élevé.

## Import des librairies

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import collections
from sklearn.metrics import accuracy_score
import numpy as np

## Chargement des données

In [2]:
x_train = pd.read_csv('../x_train.csv', index_col='ID')
y_train = pd.read_csv('../y_train.csv', index_col='ID')
train = pd.concat([x_train, y_train], axis=1)
test = pd.read_csv('../x_test.csv', index_col='ID')
train.head()

,DATE,STOCK,INDUSTRY,INDUSTRY_GROUP,SECTOR,SUB_INDUSTRY,RET_1,VOLUME_1,RET_2,VOLUME_2,...,VOLUME_16,RET_17,VOLUME_17,RET_18,VOLUME_18,RET_19,VOLUME_19,RET_20,VOLUME_20,RET
ID,,,,,,,,,,,,,,,,,,,,,
0,0,2,18,5,3,44,-0.015748,0.147931,-0.015504,0.179183,...,0.630899,0.003254,-0.379412,0.008752,-0.110597,-0.012959,0.174521,-0.002155,-0.000937,True
1,0,3,43,15,6,104,0.003984,NaN,-0.090580,NaN,...,NaN,0.003774,NaN,-0.018518,NaN,-0.028777,NaN,-0.034722,NaN,True
2,0,4,57,20,8,142,0.000440,-0.096282,-0.058896,0.084771,...,-0.010336,-0.017612,-0.354333,-0.006562,-0.519391,-0.012101,-0.356157,-0.006867,-0.308868,False
3,0,8,1,1,1,2,0.031298,-0.429540,0.007756,-0.089919,...,0.012105,0.033824,-0.290178,-0.001468,-0.663834,-0.013520,-0.562126,-0.036745,-0.631458,False
4,0,14,36,12,5,92,0.027273,-0.847155,-0.039302,-0.943033,...,-0.277083,-0.012659,0.139086,0.004237,-0.017547,0.004256,0.579510,-0.040817,0.802806,False


In [3]:
ret=[f'RET_{k}' for k in range(20,0,-1)]
vol=[f'VOLUME_{k}' for k in range(20,0,-1)]
target = 'RET'
categories = ['STOCK','SUB_INDUSTRY','INDUSTRY','INDUSTRY_GROUP','SECTOR']
features_base = ret+vol

## Paramètres

In [4]:
nb_shifts = 5 #nombre de features supplémentaires
seuil_ret = 1000
seuil_vol = 1000
rf_params = {
'n_estimators': 500,
'max_depth': 2**3,
'random_state': 0,
'n_jobs': -1
}
seuil_sector = 0
seuil_other = 36000

In [5]:
seuil = [seuil_other,seuil_other,seuil_other,seuil_other,seuil_sector]

## Nettoyage des données

In [6]:
train_cl = train.copy()
test_cl = test.copy()

for ind_groupe in np.intersect1d(train['INDUSTRY_GROUP'].unique(),test['INDUSTRY_GROUP'].unique()):
    for ret_t,vol_t in zip(ret,vol):
            med_ret = train_cl[ret_t][train['INDUSTRY_GROUP']==ind_groupe].median()
            med_vol = train_cl[vol_t][train['INDUSTRY_GROUP']==ind_groupe].median()

            train_cl.loc[:,ret_t] = train_cl.loc[:,ret_t].fillna(med_ret)
            train_cl.loc[:,vol_t] = train_cl.loc[:,vol_t].fillna(med_vol)
            test_cl.loc[:,ret_t] = test_cl.loc[:,ret_t].fillna(med_ret)
            test_cl.loc[:,vol_t] = test_cl.loc[:,vol_t].fillna(med_vol)

train_cl  = train_cl[np.all([train_cl[col]<seuil_ret for col in ret],axis=0)]
train_cl  = train_cl[np.all([train_cl[col]<seuil_vol for col in vol],axis=0)]

scaler = StandardScaler()
train_cl.loc[:,ret+vol] = scaler.fit_transform(train_cl.loc[:,ret+vol])
test_cl.loc[:,ret+vol] = scaler.fit_transform(test_cl.loc[:,ret+vol])

## Sélection des groupes à conserver selon les seuils

In [7]:
model_labels = [[] for ind_cat in range(len(categories))]

for ind_cat,cat in enumerate(categories):
    
    count_group = pd.Series(collections.Counter(train_cl[cat])).sort_index()
    for group in count_group.index:
        
        if count_group.loc[group]>=seuil[ind_cat]:
            model_labels[ind_cat].append(group)
            
    print(f"{len(model_labels[ind_cat])} modèles spécifiques de {cat} vont être entraînés")

0 modèles spécifiques de STOCK vont être entraînés
0 modèles spécifiques de SUB_INDUSTRY vont être entraînés
0 modèles spécifiques de INDUSTRY vont être entraînés
1 modèles spécifiques de INDUSTRY_GROUP vont être entraînés
12 modèles spécifiques de SECTOR vont être entraînés


## Création des new features par groupes

### Création de dataframes (vides) pour les new features

In [8]:
new_feat_train = pd.DataFrame()
new_feat_test = pd.DataFrame()
shifts = range(1,nb_shifts+1)
new_features = []
statistics = ['median']
target_feature = 'RET'

for shift in shifts:
    for stat in statistics:
        name = f'{target_feature}_{shift}_CATEGORIE_{stat}'
        new_features.append(name)
        for data in [new_feat_train,new_feat_test]:
            data[name] = None

### Calcul des new features pour chaque groupe

In [9]:
train_temp = train_cl.copy()
test_temp = test_cl.copy()
#Copie des datasets d'où on va peu à peu retirer les données après leur utilisation

for ind_cat,cat in enumerate(categories):
    for ind_group,group in enumerate(model_labels[ind_cat]):
        
        x_train_cl = train_temp.copy()[train_temp[cat]==group]
        y_train_cl = train_temp[target][train_temp[cat]==group]
        
        x_test = test_temp.copy().loc[test_temp[cat]==group,:]

        shifts = range(1,nb_shifts+1)
        statistics = ['median']
        gb_features = [categories[ind_cat],'DATE']
        target_feature = 'RET'
        
        for shift in shifts:
            for stat in statistics:
                name = f'{target_feature}_{shift}_CATEGORIE_{stat}'
                feat = f'{target_feature}_{shift}'
                for data in [x_train_cl,x_test]:
                    if len(data)>0:
                        data[name] = data.groupby(gb_features)[feat].transform(stat)
                
        new_feat_train = pd.concat([new_feat_train,x_train_cl[new_features]])
        new_feat_test = pd.concat([new_feat_test,x_test[new_features]])
        
        train_temp = train_temp[train_temp[cat]!=group]
        test_temp = test_temp[test_temp[cat]!=group] #On supprime les données utilisées

### Calcul des new features pour le modèle général si nécessaire

In [10]:
for data_temp,new_feat_data in zip([train_temp,test_temp],[new_feat_train,new_feat_test]):

    if len(data_temp)>0:

        x_data = data_temp.copy()

        shifts = range(1,nb_shifts+1)
        statistics = ['median']
        gb_features = ['SECTOR','DATE']
        target_feature = 'RET'

        for shift in shifts:
            for stat in statistics:
                name = f'{target_feature}_{shift}_CATEGORIE_{stat}'
                feat = f'{target_feature}_{shift}'
                x_data[name] = x_data.groupby(gb_features)[feat].transform(stat)

        new_feat_data = pd.concat([new_feat_data,x_data[new_features]])

### Ajout des new features aux datasets

In [11]:
new_feat_train = new_feat_train.loc[train_cl.index]
new_feat_test = new_feat_test.loc[test_cl.index] #Réindexage pour associer les new features aux bonnes lignes

train_cl[new_features] = new_feat_train
test_cl[new_features] = new_feat_test #Ajout des colonnes new features au dataset d'entraînement

## Entraînement des modèles

### Modèles par groupes

In [14]:
models = [{} for ind_cat in range(len(categories)+1)]
features = features_base + new_features

for ind_cat,cat in enumerate(categories):
    for ind_group,group in enumerate(model_labels[ind_cat]):
        
        x_train_cl = train_cl.copy()[train_cl[cat]==group]
        y_train_cl = train_cl[target][train_cl[cat]==group]

        model = RandomForestClassifier(**rf_params)
        model.fit(x_train_cl[features], y_train_cl)
        models[ind_cat][group] = model

        print(f"Modèle {cat} {group} entraîné : {len(x_train_cl)} données")

Modèle INDUSTRY_GROUP 16 entraîné : 39290 données
Modèle SECTOR 0 entraîné : 6304 données
Modèle SECTOR 1 entraîné : 21264 données
Modèle SECTOR 2 entraîné : 18967 données
Modèle SECTOR 3 entraîné : 55473 données
Modèle SECTOR 4 entraîné : 63519 données
Modèle SECTOR 5 entraîné : 17295 données
Modèle SECTOR 6 entraîné : 55122 données
Modèle SECTOR 7 entraîné : 87903 données
Modèle SECTOR 8 entraîné : 70843 données
Modèle SECTOR 9 entraîné : 5555 données
Modèle SECTOR 10 entraîné : 13295 données
Modèle SECTOR 11 entraîné : 3054 données


### Modèle général si nécessaire

In [ ]:
if len(train_temp)>0:
    
    x_train_cl = train_cl.copy()
    y_train_cl = train_cl[target]

    model = RandomForestClassifier(**rf_params)
    model.fit(x_train_cl[features], y_train_cl)
    models[5]['general'] = model
    print(f"Modèle général entraîné : {len(x_train_cl)} données")

## Prédictions

### Prédictions par groupe

In [15]:
test_temp = test_cl.copy()
y_pred = pd.Series()

for ind_cat,cat in enumerate(categories):
    for ind_group,group in enumerate(model_labels[ind_cat]):
        
        x_test = test_temp.copy().loc[test_temp[cat]==group,:]
        if len(x_test)>0:
            
            index = x_test.index
            model = models[ind_cat][group]
            y_pred = pd.concat([y_pred,pd.Series(model.predict(x_test[features]),index=x_test.index)])
            print(f"{len(x_test)} données prédites avec le modèle {cat} {group}")

            test_temp = test_temp[test_temp[cat]!=group] #On supprime les données utilisées

17939 données prédites avec le modèle INDUSTRY_GROUP 16
2177 données prédites avec le modèle SECTOR 0
11844 données prédites avec le modèle SECTOR 1
9176 données prédites avec le modèle SECTOR 2
27312 données prédites avec le modèle SECTOR 3
27951 données prédites avec le modèle SECTOR 4
7977 données prédites avec le modèle SECTOR 5
27706 données prédites avec le modèle SECTOR 6
23248 données prédites avec le modèle SECTOR 7
30092 données prédites avec le modèle SECTOR 8
3358 données prédites avec le modèle SECTOR 9
5733 données prédites avec le modèle SECTOR 10
3916 données prédites avec le modèle SECTOR 11


### Prédictions avec le modèle général si nécessaire

In [16]:
if len(test_temp)>0:
    x_test = test_temp.copy()
    index = x_test.index
    model = models[5]['general']
    
    y_pred = pd.concat([y_pred,pd.Series(model.predict(x_test[features]),index=x_test.index)])
    print(f"{len(x_test)} données prédites avec le modèle général")

## Préparation des données prédites pour submission

In [17]:
y_pred = y_pred[test_cl.index]
y_pred.name = target
# y_pred.to_csv('./nom_submit.csv', index=True, header=True)